In [5]:
import chromadb
#client = chromadb.PersistentClient(path="C:\\Users\\Leo\\실습\\chroma_db")
client = chromadb.PersistentClient(path='./data')
collection = client.get_or_create_collection("test_data01")

collection.add(
    ids = ["doc1", "doc2"],
    embeddings= [
        [0.1, 0.2, 0.3], 
        [0.4, 0.5, 0.6]
    ],
    metadatas= [
        {"text": "안녕하세요"}, 
        {"text": "날씨가 좋네요"}
    ],

)

In [6]:
query = [[0.1, 0.2, 0.3]]
result = collection.query(query_embeddings=query, n_results=1)
print(result)

{'ids': [['doc1']], 'embeddings': None, 'documents': [[None]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'text': '안녕하세요'}]], 'distances': [[0.0]]}


In [7]:
result_text = result['metadatas'][0][0]['text']
print("가장 유사한 문장:", result_text)

가장 유사한 문장: 안녕하세요


In [8]:
result2 = collection.query(query_embeddings=query, n_results=1, include=['embeddings', 'metadatas'])
result2

{'ids': [['doc1']],
 'embeddings': [array([[0.1       , 0.2       , 0.30000001]])],
 'documents': None,
 'uris': None,
 'included': ['embeddings', 'metadatas'],
 'data': None,
 'metadatas': [[{'text': '안녕하세요'}]],
 'distances': None}

In [9]:
all_data = collection.get()
all_data

{'ids': ['doc1', 'doc2'],
 'embeddings': None,
 'documents': [None, None],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'text': '안녕하세요'}, {'text': '날씨가 좋네요'}]}

In [10]:
collection.get(ids = ["doc1"])

{'ids': ['doc1'],
 'embeddings': None,
 'documents': [None],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'text': '안녕하세요'}]}

In [11]:
collection.add(
    ids = ["doc3"],
    embeddings = [
        [0.7, 0.8, 0.9]
    ],
    metadatas = [
        {"text": "안녕히 가세요"}
    ],)
all_data = collection.get()
all_data

{'ids': ['doc1', 'doc2', 'doc3'],
 'embeddings': None,
 'documents': [None, None, None],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'text': '안녕하세요'}, {'text': '날씨가 좋네요'}, {'text': '안녕히 가세요'}]}

In [12]:
collection.delete(ids=["doc3"])
all_data = collection.get(include=['embeddings', 'metadatas'])
all_data

{'ids': ['doc1', 'doc2'],
 'embeddings': array([[0.1       , 0.2       , 0.30000001],
        [0.40000001, 0.5       , 0.60000002]]),
 'documents': None,
 'uris': None,
 'included': ['embeddings', 'metadatas'],
 'data': None,
 'metadatas': [{'text': '안녕하세요'}, {'text': '날씨가 좋네요'}]}

In [13]:
import pandas as pd
df = pd.DataFrame({
    "ids": all_data["ids"],
    "embeddings": [embedding for embedding in all_data["embeddings"]],
    "texts": [metadata['text'] for metadata in all_data["metadatas"]]
})


df.head()

,ids,embeddings,texts
0,doc1,"[0.10000000149011612, 0.20000000298023224, 0.3...",안녕하세요
1,doc2,"[0.4000000059604645, 0.5, 0.6000000238418579]",날씨가 좋네요


In [23]:
#[기존 데이터 임베딩 후 chromaDB 삽입]

import chroma_db
import pandas as pd
from sentence_transformers import SentenceTransformer
df = pd.read_csv('C:\\Users\\Leo\\code\\ch06_chromaDB\\test02\\test_file.csv', encoding='utf8')
df

,sentence
0,안녕하세요.
1,누구세요?
2,안녕히 가세요.


In [18]:
sc_list = df['sentence'].tolist()
sc_list

['안녕하세요.', '누구세요?', '안녕히 가세요.']

In [24]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
ids_list = []
ebd_list = []
text_list = []
ith = 0
for sc in sc_list:
    idx = f"doc{ith+1}"
    ids_list.append(idx)
    ith += 1

    text_doc = {"text": sc}
    text_list.append(text_doc)

    embedding = model.encode(sc).tolist()
    ebd_list.append(embedding)

ids_list

['doc1', 'doc2', 'doc3']

In [21]:
print(ebd_list)
print(text_list)

[[-0.05254364758729935, 0.1017642691731453, 0.02275516651570797, -0.0025180845987051725, -0.030778618529438972, -0.03499298915266991, 0.08704984188079834, 0.048225823789834976, -0.035652320832014084, -0.029717668890953064, 0.08730760216712952, -0.006052766460925341, 0.037013567984104156, -0.06087620183825493, 0.06154434010386467, -0.03621429577469826, 0.00882658176124096, 0.012834347784519196, -0.06590220332145691, -0.060595326125621796, 0.05625663697719574, 0.031584590673446655, 0.0483044758439064, 0.014921875670552254, -0.05143671855330467, -0.01982727088034153, 0.01773010566830635, 0.044114042073488235, 0.16151192784309387, 0.014964550733566284, 0.013138845562934875, 0.05499523878097534, 0.07119585573673248, 0.03456219658255577, -0.03526632860302925, 0.015923647210001945, -0.04928414523601532, 0.004164614249020815, -0.02603912726044655, 0.01747170463204384, -0.15378086268901825, -0.05232534557580948, 0.05868741124868393, -0.11337690055370331, 0.06140722334384918, -0.0758355781435966

In [26]:
import chromadb
client = chromadb.PersistentClient(path='./data')
collection = client.get_or_create_collection("test_data02")
collection.add(
    ids = ids_list,
    embeddings= ebd_list,
    metadatas= text_list,
)

In [6]:
# [Chunk]
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

with open("./doc.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

raw_text


'Etching 공정 전에는 반드시 세정 공정이 완료되어야 합니다. PM 장비의 필터는 주 1회 정기적으로 교체해야 합니다. 웨이퍼 투입 전 챔버 내부의 온도 안정화가 필요합니다. 불량률이 2%를 초과할 경우 원인 분석 보고서를 제출해야 합니다. 클린룸 입장 전에는 반드시 정전기 방지복을 착용해야 합니다. 포토 공정 시 PR 코팅 두께는 1.5μm 이상 유지해야 합니다. 검사 장비는 매일 초기화 후 기능 점검을 실시합니다. 주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다. X선 검사 장비는 비정상 신호 발생 시 즉시 사용 중지합니다. 로더 설비의 진공 펌프는 월 1회 이상 윤활 상태를 점검합니다. 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다. 동일 Lot 내에서 불량이 5개 이상 발생하면 전수 검사 실시합니다. 물류 이송 로봇은 경로 장애 감지 시 자동 우회합니다. 이온 주입 공정 후 열처리 시간은 최소 30분 이상 확보합니다. 클린룸 청소는 일 2회, 장비 청소는 주 1회 이상 실시합니다. 신입 엔지니어는 공정별 교육을 모두 이수한 후 장비 조작이 가능합니다. 공정 레시피 변경 시 Change History 문서 작성을 필수로 합니다. 수율 개선안은 월간 품질 회의에서 발표되어야 합니다. Mask alignment 오류가 0.2μm를 초과하면 장비 점검을 실시합니다. MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다. 웨이퍼 저장 캐리어는 주기적으로 오염도를 측정해야 합니다. 장비 재가동 시 Warm-up 절차를 반드시 이행합니다. Test wafer는 생산 wafer 투입 전에 반드시 시뮬레이션합니다. 제조 Lot ID는 공정마다 바코드로 자동 추적됩니다. 소자 특성 분석 결과는 전자문서 시스템에 등록합니다. 야간조 작업자는 작업 시작 전 점검 체크리스트를 반드시 확인합니다. 공정 조건이 기준에서 벗어난 경우 자동 알람이 발생합니다. 생산 중단 요청은 생산 관리팀 승인 후 진행할 수 있습니다. 사내 기술 포럼 자료는 R&D 인트라넷에 

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, 
    chunk_overlap=50, 
    separators=["\n", ".", " "]
)

chunks = text_splitter.create_documents([raw_text])
chunks

[Document(metadata={}, page_content='Etching 공정 전에는 반드시 세정 공정이 완료되어야 합니다. PM 장비의 필터는 주 1회 정기적으로 교체해야 합니다. 웨이퍼 투입 전 챔버 내부의 온도 안정화가 필요합니다. 불량률이 2%를 초과할 경우 원인 분석 보고서를 제출해야 합니다. 클린룸 입장 전에는 반드시 정전기 방지복을 착용해야 합니다. 포토 공정 시 PR 코팅 두께는 1.5μm 이상 유지해야 합니다. 검사 장비는 매일 초기화 후 기능 점검을 실시합니다. 주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다'),
 Document(metadata={}, page_content='. 주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다. X선 검사 장비는 비정상 신호 발생 시 즉시 사용 중지합니다. 로더 설비의 진공 펌프는 월 1회 이상 윤활 상태를 점검합니다. 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다. 동일 Lot 내에서 불량이 5개 이상 발생하면 전수 검사 실시합니다. 물류 이송 로봇은 경로 장애 감지 시 자동 우회합니다. 이온 주입 공정 후 열처리 시간은 최소 30분 이상 확보합니다. 클린룸 청소는 일 2회, 장비 청소는 주 1회 이상 실시합니다'),
 Document(metadata={}, page_content='. 클린룸 청소는 일 2회, 장비 청소는 주 1회 이상 실시합니다. 신입 엔지니어는 공정별 교육을 모두 이수한 후 장비 조작이 가능합니다. 공정 레시피 변경 시 Change History 문서 작성을 필수로 합니다. 수율 개선안은 월간 품질 회의에서 발표되어야 합니다. Mask alignment 오류가 0.2μm를 초과하면 장비 점검을 실시합니다. MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다. 웨이퍼 저장 캐리어는 주기적으로 오염도를 측정해야 합니다. 장비 재가동 시 Warm-up 절차를 반드시 이행합니다'),
 Document(metadata={}, page_content='. 장비 재

In [10]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chromaDB_chunked"
)

vectorstore = Chroma(
    persist_directory="./chromaDB_chunked",
    embedding_function=embedding_model
)
collection = vectorstore._collection
docs = collection.get()
docs

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\User\.conda\envs\GenAI2503\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'ids': ['fd35cdfa-a1c8-4af6-9663-ef25ee02ebfe',
  '8a8c98da-6b1e-4295-8e89-773dc7f9eb47',
  'e3605739-9702-47cd-994a-99b12ed803ca',
  'a97f0d9b-57a9-41b7-9c6d-d05db7e18774',
  '1911ca0c-9142-4e50-98de-098e612c4296'],
 'embeddings': None,
 'documents': ['Etching 공정 전에는 반드시 세정 공정이 완료되어야 합니다. PM 장비의 필터는 주 1회 정기적으로 교체해야 합니다. 웨이퍼 투입 전 챔버 내부의 온도 안정화가 필요합니다. 불량률이 2%를 초과할 경우 원인 분석 보고서를 제출해야 합니다. 클린룸 입장 전에는 반드시 정전기 방지복을 착용해야 합니다. 포토 공정 시 PR 코팅 두께는 1.5μm 이상 유지해야 합니다. 검사 장비는 매일 초기화 후 기능 점검을 실시합니다. 주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다',
  '. 주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다. X선 검사 장비는 비정상 신호 발생 시 즉시 사용 중지합니다. 로더 설비의 진공 펌프는 월 1회 이상 윤활 상태를 점검합니다. 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다. 동일 Lot 내에서 불량이 5개 이상 발생하면 전수 검사 실시합니다. 물류 이송 로봇은 경로 장애 감지 시 자동 우회합니다. 이온 주입 공정 후 열처리 시간은 최소 30분 이상 확보합니다. 클린룸 청소는 일 2회, 장비 청소는 주 1회 이상 실시합니다',
  '. 클린룸 청소는 일 2회, 장비 청소는 주 1회 이상 실시합니다. 신입 엔지니어는 공정별 교육을 모두 이수한 후 장비 조작이 가능합니다. 공정 레시피 변경 시 Change History 문서 작성을 필수로 합니다. 수율 개선안은 월간 품질 회의에서 발표되어야 합니다. Mask alignment 오류가 0.2μm를 초과하면 장비 점검을 실

In [11]:
docs = vectorstore._collection.get(limit=5)
for i in range(len(docs["documents"])):
    print(f"Document {i+1}: {docs['documents'][i]}")
    print(f"Metadata {i+1}: {docs['metadatas'][i]}")
    print("-----")

Document 1: Etching 공정 전에는 반드시 세정 공정이 완료되어야 합니다. PM 장비의 필터는 주 1회 정기적으로 교체해야 합니다. 웨이퍼 투입 전 챔버 내부의 온도 안정화가 필요합니다. 불량률이 2%를 초과할 경우 원인 분석 보고서를 제출해야 합니다. 클린룸 입장 전에는 반드시 정전기 방지복을 착용해야 합니다. 포토 공정 시 PR 코팅 두께는 1.5μm 이상 유지해야 합니다. 검사 장비는 매일 초기화 후 기능 점검을 실시합니다. 주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다
Metadata 1: None
-----
Document 2: . 주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다. X선 검사 장비는 비정상 신호 발생 시 즉시 사용 중지합니다. 로더 설비의 진공 펌프는 월 1회 이상 윤활 상태를 점검합니다. 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다. 동일 Lot 내에서 불량이 5개 이상 발생하면 전수 검사 실시합니다. 물류 이송 로봇은 경로 장애 감지 시 자동 우회합니다. 이온 주입 공정 후 열처리 시간은 최소 30분 이상 확보합니다. 클린룸 청소는 일 2회, 장비 청소는 주 1회 이상 실시합니다
Metadata 2: None
-----
Document 3: . 클린룸 청소는 일 2회, 장비 청소는 주 1회 이상 실시합니다. 신입 엔지니어는 공정별 교육을 모두 이수한 후 장비 조작이 가능합니다. 공정 레시피 변경 시 Change History 문서 작성을 필수로 합니다. 수율 개선안은 월간 품질 회의에서 발표되어야 합니다. Mask alignment 오류가 0.2μm를 초과하면 장비 점검을 실시합니다. MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다. 웨이퍼 저장 캐리어는 주기적으로 오염도를 측정해야 합니다. 장비 재가동 시 Warm-up 절차를 반드시 이행합니다
Metadata 3: None
-----
Document 4: . 장비 재가동 시 Warm-up 절차를 반드시 이행합니다. Test wafer는 

In [14]:
docs = vectorstore._collection.get(limit=3, include=['embeddings', 'documents'])
for i, embedding in enumerate(docs['embeddings']):
    print(f"\ndocument {i+1} emdedding vector(길이: {len(embedding)}):")
    print(embedding[:10], "...")
    print("-----")


document 1 emdedding vector(길이: 384):
[ 0.01478547  0.06491547  0.08781839 -0.0091751  -0.03898884 -0.04902731
  0.02531443  0.04685926 -0.06610113 -0.00950357] ...
-----

document 2 emdedding vector(길이: 384):
[-0.00391315 -0.00193419  0.04884215  0.00527482 -0.02369014 -0.03099549
  0.06199988  0.06676459 -0.02932272 -0.0264822 ] ...
-----

document 3 emdedding vector(길이: 384):
[ 0.01213797  0.06109216  0.05667148 -0.01892162  0.00030271 -0.02973588
  0.0009332  -0.01253839  0.00214408 -0.00773625] ...
-----
